In [ ]:
from functools import lru_cache
import random
from typing import Literal, TypeAlias, Union
import numpy as np
import pygame
from pprint import pprint as print


# Define the Task class
class Task:
    def __init__(self, id: int, duration: int, priority: int):
        self.id = id
        self.duration = duration
        self.priority = priority

    def __repr__(self):
        return f"Task(id={self.id}, duration={self.duration}, priority={self.priority})"

    @property
    def name(self):
        return f"T{self.id}({self.duration}:{self.priority})"


# Define the Resource class
class Resource:
    def __init__(self, id: int, capacity: int):
        self.id = id
        self.capacity = capacity

    def __repr__(self):
        return f"Resource(id={self.id}, capacity={self.capacity})"


# Chromosome representation
class Chromosome:
    def __init__(self, gene: list[int], fitness: int = 0):
        self.gene = gene  # Gene is a list of task assignments to resources
        self.fitness = fitness

    def __lt__(self, other: "Chromosome"):
        return self.fitness < other.fitness


# Fitness function
def calculate_fitness(
    chromosome: Chromosome, tasks: list[Task], resources: list[Resource]
) -> float:
    # Calculate total execution time and priority score
    resource_times: dict[int, float] = {r.id: 0 for r in resources}
    priority_score = 0

    for task_idx, resource_id in enumerate(chromosome.gene):
        task = tasks[task_idx]
        resource_times[resource_id] += task.duration
        priority_score += task.priority

    makespan = max(resource_times.values())
    load_balance = np.std(list(resource_times.values()))

    # Objective: Minimize makespan and load balance, maximize priority score
    fitness = (
        (1 / makespan) + (1 / (1 + load_balance)) + (priority_score / (len(tasks) * 5))
    )
    return fitness


# Selection operator
def selection(population: list[Chromosome]) -> tuple[Chromosome, Chromosome]:
    # Using tournament selection
    tournament_size = 3
    # select random chromosomes and return the best two
    selected = random.sample(population, tournament_size)
    selected.sort(reverse=True)
    return (selected[0], selected[1])


# Crossover operator
def crossover(parent1: Chromosome, parent2: Chromosome) -> Chromosome:
    crossover_point = random.randint(1, len(parent1.gene) - 2)
    child_gene = parent1.gene[:crossover_point] + parent2.gene[crossover_point:]
    return Chromosome(gene=child_gene)


# Mutation operator
def mutation(chromosome: Chromosome, num_resources: int, mutation_rate: float = 0.01):
    for i in range(len(chromosome.gene)):
        if random.random() < mutation_rate:
            chromosome.gene[i] = random.randint(0, num_resources - 1)


# Initialize Pygame
pygame.init()
# get display info
infoObject = pygame.display.Info()
SCREEN_WIDTH, SCREEN_HEIGHT = infoObject.current_w, (infoObject.current_h * 0.6)

# Define margins as a percentage of screen size
margin_percentage = 0.05  # 5% margin on each side
margin_x = int(SCREEN_WIDTH * margin_percentage)
margin_y = int(SCREEN_HEIGHT * (margin_percentage - 0.02))

# Calculate WIDTH and HEIGHT based on screen size and margins
WIDTH = SCREEN_WIDTH - 2 * margin_x
HEIGHT = SCREEN_HEIGHT - 2 * margin_y
# Set screen dimensions
# WIDTH, HEIGHT = 3300, 800
SCREEN = pygame.display.set_mode((WIDTH, HEIGHT))
# Display title
pygame.display.set_caption("Task Scheduling Visualization")
# Get font
FONT = pygame.font.SysFont("Arial", 16)
BOLD_FONT = pygame.font.SysFont("Arial", 16, bold=True)
BOLD_FONT_TITLE = pygame.font.SysFont("Arial", 22, bold=True)
# Set clock pedding

ScheduleResType: TypeAlias = int
ScheduleKeysType: TypeAlias = Literal["Task", "Start", "Finish"]
ScheduleValueType: TypeAlias = Union[Task, int]
ScheduleReturnType: TypeAlias = dict[
    ScheduleResType, list[dict[ScheduleKeysType, ScheduleValueType]]
]


# Function to create the schedule with start times for visualization
def create_schedule(
    chromosome: Chromosome, tasks: list[Task], resources: list[Resource]
) -> ScheduleReturnType:
    schedule = {r.id: [] for r in resources}
    resource_end_times = {r.id: 0 for r in resources}

    for task_idx, resource_id in enumerate(chromosome.gene):
        task = tasks[task_idx]
        start_time = resource_end_times[resource_id]
        finish_time = start_time + task.duration
        schedule[resource_id].append(
            {
                "Task": task,
                "Start": start_time,
                "Finish": finish_time,
            }
        )
        resource_end_times[resource_id] = finish_time

    return schedule


@lru_cache(maxsize=None)
def create_color(_task_id: str) -> tuple[int, int, int]:
    color = (random.randint(20, 255), random.randint(30, 255), random.randint(40, 255))

    match color:
        # recreate the color if it is too light
        case (r, g, b) if r + g + b > 760:
            return create_color(_task_id)
        case _:
            return color


# Function to draw the schedule using Pygame
def draw_schedule(
    schedule: ScheduleReturnType,
    generation: int,
    best_fitness: float,
    total_duration: int,
    gen_without_improvement: int = 0,
):
    margin = 130
    BORDER_RGB = (11, 37, 87)
    SCREEN.fill((255, 255, 255))
    resource_height = (HEIGHT - 2 * margin) // (len(schedule) + 1)
    max_time = max(
        [task_info["Finish"] for tasks in schedule.values() for task_info in tasks]
    )
    time_scale = (WIDTH - 2 * margin) / (max_time + 1)

    default = (True, (0, 0, 0))

    # Draw a border around the schedule
    pygame.draw.rect(
        surface=SCREEN,
        color=BORDER_RGB,
        rect=(margin, margin, WIDTH - 2 * margin + 100, HEIGHT - 2 * margin),
        width=4,
        border_radius=6,
    )

    for idx, (resource_id, tasks_info) in enumerate(schedule.items()):
        y = margin + (idx + 1) * resource_height
        # ADD BOLD RESOURCE LABEL HERE
        resource_label = BOLD_FONT.render(f"Resource {resource_id}", *default)
        SCREEN.blit(resource_label, (margin + 8, y - resource_height // 5 ))

        for task_info in tasks_info:
            task = task_info["Task"]
            start = task_info["Start"]
            duration = task.duration
            x = margin + start * time_scale
            width = duration * time_scale

            color = create_color(task.name)

            pygame.draw.rect(
                surface=SCREEN,
                color=color,
                rect=(x + 100, y - resource_height // 2, width, resource_height - 10),
                border_radius=6,
            )
            task_label = BOLD_FONT.render(task.name, True, (255, 255, 255))
            SCREEN.blit(source=task_label, dest=(x + 105, y - resource_height // 2 + 5))

    # Display generation and fitness
    # rectangle for generation and fitness
    pygame.draw.rect(
        surface=SCREEN,
        color=BORDER_RGB,
        rect=(margin, 16, 500, 100),
        width=4,
        border_radius=6,
    )

    gen_text = BOLD_FONT_TITLE.render(f"Generation: {generation + 1}", *default)
    fitness_text = BOLD_FONT_TITLE.render(f"Best Fitness: {best_fitness:.4f}", *default)

    total_tasks = sum([len(tasks) for tasks in schedule.values()])
    total_text = BOLD_FONT_TITLE.render(f"Task Total: {total_tasks}", *default)


    pygame.draw.rect(
        surface=SCREEN,
        color=BORDER_RGB,
        rect=(margin + 600, 16, 500, 100),
        width=4,
        border_radius=6,
    )
    gen2_text = BOLD_FONT_TITLE.render(
        f"Generation without improvement: {gen_without_improvement}", *default
    )
    total_duration_text = BOLD_FONT_TITLE.render(f"Total duration: {total_duration} secs", *default)
    mean_time_text = BOLD_FONT_TITLE.render(f"Mean time by tasks: {total_duration / total_tasks:.2f} secs", *default)

    TEXT_MARGIN = 20
    SCREEN.blit(gen_text, (margin + TEXT_MARGIN, 20))
    SCREEN.blit(fitness_text, (margin + TEXT_MARGIN, 50))
    SCREEN.blit(gen2_text, (margin  + TEXT_MARGIN, 80))

    SCREEN.blit(total_text, (margin + 600 + TEXT_MARGIN, 20))
    SCREEN.blit(total_duration_text, (margin + 600 + TEXT_MARGIN, 80))
    SCREEN.blit(mean_time_text, (margin + 600 + TEXT_MARGIN, 50))

    tasks_by_duration_count = {}
    for resource_id, tasks_info in schedule.items():
        for task_info in tasks_info:
            task = task_info["Task"]
            if task.duration not in tasks_by_duration_count:
                tasks_by_duration_count[task.duration] = 0
            tasks_by_duration_count[task.duration] += 1
    
    tasks_by_duration_count = dict(sorted(tasks_by_duration_count.items()))

    # Define starting positions and calculate dynamic spacing for horizontal layout
    num_durations = len(tasks_by_duration_count)
    rectangle_width = 300
    rectangle_height = 30

    # Total width occupied by all rectangles
    total_rect_width = num_durations * rectangle_width

    # Calculate available space for spacing
    available_space = WIDTH - 2 * margin - total_rect_width

    # Calculate spacing between rectangles
    if num_durations > 1:
        spacing_x = available_space / (num_durations + 1)
    else:
        spacing_x = available_space / 2  # Center the single rectangle

    # Enforce minimum spacing if necessary
    min_spacing_x = 20  # Minimum spacing in pixels
    spacing_x = max(spacing_x, min_spacing_x)

    # Starting x position
    rectangle_start_x = margin + spacing_x

    # y position remains the same (placed near the bottom)
    rectangle_y = HEIGHT - 100

    for idx, (duration, count) in enumerate(tasks_by_duration_count.items()):
        # Render the duration count text in bold
        duration_text = BOLD_FONT_TITLE.render(f"Duration {duration} secs: {count}", True, (255, 255, 255))
        text_width, _ = duration_text.get_size()
        
        # Define rectangle position for horizontal layout
        rectangle_x = rectangle_start_x + idx * (rectangle_width + spacing_x)
        
        # Draw the rectangle
        pygame.draw.rect(
            surface=SCREEN,
            color=create_color(f"Duration {duration}"),
            rect=(rectangle_x, rectangle_y, rectangle_width, rectangle_height),
            # width=4,
            border_radius=6,
        )
        
        # Calculate centered text position below the rectangle
        text_x = rectangle_x + (rectangle_width - text_width) / 2
        text_y = rectangle_y
        
        # Blit the duration text onto the screen
        SCREEN.blit(duration_text, (text_x, text_y))



    pygame.display.update()


# Genetic Algorithm function with Pygame visualization
def genetic_algorithm(
    tasks: list[Task], resources: list[Resource], population_size: int, generations: int
):
    num_tasks = len(tasks)
    num_resources = len(resources)
    total_duration = sum([task.duration for task in tasks])

    # Initialize population
    population: list[Chromosome] = []

    gen_without_improvement = 0
    last_best_fitness = 0

    for _ in range(population_size):
        gene = [random.randint(0, num_resources - 1) for _ in range(num_tasks)]
        chromosome = Chromosome(gene)
        chromosome.fitness = calculate_fitness(chromosome, tasks, resources)
        population.append(chromosome)

    # Evolution process
    for gen in range(generations):
        new_population: list[Chromosome] = []

        # Elitism: Keep the best chromosome
        population.sort(reverse=True)
        new_population.append(population[0])

        while len(new_population) < population_size:
            # Selection
            parent1, parent2 = selection(population=population)

            # Crossover
            child = crossover(parent1=parent1, parent2=parent2)

            # Mutation
            mutation(chromosome=child, num_resources=num_resources)

            # Calculate fitness
            child.fitness = calculate_fitness(
                chromosome=child, tasks=tasks, resources=resources
            )
            new_population.append(child)

        population = new_population

        best_chromosome = max(population, key=lambda c: c.fitness)

        if best_chromosome.fitness <= last_best_fitness:
            gen_without_improvement += 1
        else:
            gen_without_improvement = 0

        last_best_fitness = best_chromosome.fitness

        # Handle Pygame events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return best_chromosome

        # Create schedule with start and finish times
        schedule = create_schedule(best_chromosome, tasks, resources)

        # Draw the schedule
        draw_schedule(
            schedule=schedule,
            generation=gen,
            best_fitness=best_chromosome.fitness,
            total_duration=total_duration,
            gen_without_improvement=gen_without_improvement,
        )

        # Control the speed of visualization
        pygame.time.delay(100)

    pygame.time.delay(10000)
    pygame.quit()
    return best_chromosome


GENERATIONS = 200
POPULATION_SIZE = 50
MUTATION_RATE = 0.01

NUM_TASKS = 100
NUM_RESOURCES = 5
RESOURCE_CAPACITY = 10


# Main function
def main():
    tasks = generate_tasks(NUM_TASKS)
    resources = generate_resources(NUM_RESOURCES, RESOURCE_CAPACITY)

    print("Running Genetic Algorithm...")
    print(f"Tasks: {tasks}")
    print(f"Resources: {resources}")

    best_chromosome = genetic_algorithm(
        tasks=tasks,
        resources=resources,
        population_size=POPULATION_SIZE,
        generations=GENERATIONS,
    )

    print(f"Best Fitness = {best_chromosome.fitness:.4f}")
    print(f"Best Gene = {best_chromosome.gene}")


if __name__ == "__main__":
    # Generate sample tasks
    def generate_tasks(num_tasks: int):
        tasks = []
        for task_id in range(num_tasks):
            # Task duration between 1 and 10 units
            duration = random.randint(2, 10)
            # Priority between 1 (lowest) and 5 (highest)
            priority = random.randint(1, 5)
            tasks.append(Task(id=task_id, duration=duration, priority=priority))
        return tasks

    # Generate sample resources
    def generate_resources(num_resources: int, capacity: int):
        return [Resource(id=id, capacity=capacity) for id in range(num_resources)]

    try:
        main()
    except Exception as e:
        print(e)
        pygame.quit()
        raise e

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default


'Running Genetic Algorithm...'
('Tasks: [Task(id=0, duration=7, priority=2), Task(id=1, duration=2, '
 'priority=2), Task(id=2, duration=4, priority=5), Task(id=3, duration=3, '
 'priority=5), Task(id=4, duration=6, priority=4), Task(id=5, duration=5, '
 'priority=3), Task(id=6, duration=5, priority=1), Task(id=7, duration=10, '
 'priority=4), Task(id=8, duration=7, priority=1), Task(id=9, duration=9, '
 'priority=3), Task(id=10, duration=6, priority=4), Task(id=11, duration=7, '
 'priority=2), Task(id=12, duration=9, priority=2), Task(id=13, duration=4, '
 'priority=2), Task(id=14, duration=5, priority=2), Task(id=15, duration=5, '
 'priority=3), Task(id=16, duration=5, priority=3), Task(id=17, duration=6, '
 'priority=3), Task(id=18, duration=4, priority=3), Task(id=19, duration=6, '
 'priority=3), Task(id=20, duration=6, priority=5), Task(id=21, duration=10, '
 'priority=3), Task(id=22, duration=7, priority=1), Task(id=23, duration=9, '
 'priority=5), Task(id=24, duration=8, priorit